<a href="https://colab.research.google.com/github/Kenethoriga/FMCG_delivery_optimization/blob/main/Smart_pricing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import pandas as pd
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Point
url = 'https://raw.githubusercontent.com/Kenethoriga/FMCG_delivery_optimization/main/dataset.csv'
response = requests.get(url)
with open('dataset.csv', 'wb') as file:
    file.write(response.content)

url = 'https://raw.githubusercontent.com/Kenethoriga/FMCG_delivery_optimization/main/territories_centroids.csv'
response = requests.get(url)
with open('territories_centroids.csv', 'wb') as file:
    file.write(response.content)


In [2]:
# Load datasets
shops = pd.read_csv('dataset.csv')
centroids = pd.read_csv('territories_centroids.csv')

# Merge datasets based on territory
data = pd.merge(shops, centroids, on='territory')
data


,order_id,customer_number,transaction_date,description,total_qty,net_total,loyalty_amount,loyalty_points,discount_amount,customer_group,territory,delivery_date,delivery_window_end_time,delivery_window_start_time,set_warehouse,duka_latitude,duka_longitude,id,warehouse_latitude,warehouse_longitude
0,04aeddd4-d9b3-4cca-ab3b-072ca177a3e1,ID_172101,28/03/2022,Steam Energy Energy Drink 400ML CARTON (1.0 Item),6,2231.57,0,0,66.38,Individual,Meru,29/03/2022,19:00:00,13:00:00,Meru Main - KDKE,0.036784,37.654155,12,-1.277078,36.814669
1,04aeddd4-d9b3-4cca-ab3b-072ca177a3e1,ID_172101,28/03/2022,Sumo Candles 8PC CARTON (12.0 Outers),6,2231.57,0,0,66.38,Individual,Meru,29/03/2022,19:00:00,13:00:00,Meru Main - KDKE,0.036784,37.654155,12,-1.277078,36.814669
2,04aeddd4-d9b3-4cca-ab3b-072ca177a3e1,ID_172101,28/03/2022,Rhino Matches Kubwa 10 Boxes OUTER (1.0 Item),6,2231.57,0,0,66.38,Individual,Meru,29/03/2022,19:00:00,13:00:00,Meru Main - KDKE,0.036784,37.654155,12,-1.277078,36.814669
3,4da580db-3b2c-4dff-a181-9c1341e05ea5,ID_172101,12/03/2022,Dawn Tissue Rolls Smart 40PC BALE (1.0 Bale),5,3637.91,0,0,38.53,Individual,Meru,14/03/2022,14:00:00,08:00:00,Meru Main - KDKE,0.036784,37.654155,12,-1.277078,36.814669
4,4da580db-3b2c-4dff-a181-9c1341e05ea5,ID_172101,12/03/2022,Sawa Bath Soap 125G White DOZEN (1.0 Dzn),5,3637.91,0,0,38.53,Individual,Meru,14/03/2022,14:00:00,08:00:00,Meru Main - KDKE,0.036784,37.654155,12,-1.277078,36.814669
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43713,4802cfab-74e0-4813-8b1f-2931f068f65f,ID_066466,15/03/2022,Camel Maize Flour 1KG BALE (20.0 PC),2,1194.83,0,0,14.00,Individual,Karatina,16/03/2022,14:00:00,08:00:00,Karatina Main - KDKE,-0.564060,37.074678,6,-0.473139,37.126222
43714,5ff62a4e-d369-4e50-b049-9a1021792241,ID_066466,21/02/2022,Local White Sugar 50KG BAG (1.0 Item),4,6874.52,0,0,80.55,Individual,Karatina,22/02/2022,14:00:00,08:00:00,Karatina Main - KDKE,-0.564060,37.074678,6,-0.473139,37.126222
43715,5ff62a4e-d369-4e50-b049-9a1021792241,ID_066466,21/02/2022,Dawn Pekee Tissue Rolls Jumbo BALE (1.0 Item),4,6874.52,0,0,80.55,Individual,Karatina,22/02/2022,14:00:00,08:00:00,Karatina Main - KDKE,-0.564060,37.074678,6,-0.473139,37.126222
43716,5ff62a4e-d369-4e50-b049-9a1021792241,ID_066466,21/02/2022,Dawn Pekee Tissue Rolls 40PC BALE (1.0 Item),4,6874.52,0,0,80.55,Individual,Karatina,22/02/2022,14:00:00,08:00:00,Karatina Main - KDKE,-0.564060,37.074678,6,-0.473139,37.126222


In [6]:
# Extract the relevant column
order_amounts = data['net_total']
order_amounts

0        2231.57
1        2231.57
2        2231.57
3        3637.91
4        3637.91
          ...   
43713    1194.83
43714    6874.52
43715    6874.52
43716    6874.52
43717    6874.52
Name: net_total, Length: 43718, dtype: float64

In [7]:
import numpy as np
import scipy.stats as stats

def calculate_confidence_interval(data, confidence=0.95):
    # Convert data to a NumPy array if it's not already
    data = np.array(data)
    mean = np.mean(data)
    sem = stats.sem(data)
    interval = sem * stats.t.ppf((1 + confidence) / 2., len(data)-1)
    return mean, mean - interval, mean + interval

# Calculate the confidence interval for the order_amounts
mean, lower, upper = calculate_confidence_interval(order_amounts)
print(f"Mean: {mean}, 95% CI: ({lower}, {upper})")


Mean: 10464.409966375406, 95% CI: (10180.996546810602, 10747.82338594021)
